## Imports

In [ ]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image

## Load_YoloV8_Model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = YOLO(r'runs\detect\train\weights\best.pt')  # load a custom model
# model.predict(r"..\TruckDataset\train\images\time3_frame5_003.jpg", save=True, imgsz=640, conf=0.5, show=True)
model.to(device)

## Detection__Image

In [ ]:
results = model(r"..\TruckDataset\valid\images\time1_frame_142.jpg")
print('\n output image shape: ', results[0].plot().shape)

print('\n NUM of Trucks in Frame: ', results[0].boxes.shape[0])
im_array = results[0].plot()  # plot a BGR numpy array of predictions

cv2.putText(im_array,f'Yolov8 Model - truck: {results[0].boxes.shape[0]}',(50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
im.show()

## Detection_Video_MultiTime

In [ ]:
# Dictionary of different times
# a time have a start and end time 
# each of start and end time is a list: [h, m, s]
## you can comment some of times in dictinary if have short time
times = {'time1' : [[4, 56, 6], [4, 56, 30]],
         'time2' : [[5, 51, 33], [5, 53, 3]],
         'time3' : [[12, 4, 19], [12, 7, 0]],
         'time4' : [[15, 10, 10], [15, 10, 40]],
         'time5' : [[15, 21, 40], [15, 22, 26]],
         'time6' : [[16, 27, 23], [16, 28, 20]],
         'time7' : [[17, 24, 13], [17, 25, 0]],
         'time8' : [[18, 1, 30], [18, 2, 30]],
         'time9' : [[19, 14, 20], [19, 15, 13]]
         }
# times = {'time1' : [[4, 56, 6], [4, 58, 14]],
#          'time2' : [[5, 51, 33], [5, 53, 3]],
#          'time3' : [[6, 46, 22], [6, 43, 18]],
#          'time4' : [[7, 42, 41], [7, 43, 33]],
#          'time5' : [[12, 4, 19], [12, 15, 0]],
#          'time6' : [[15, 10, 10], [15, 10, 40]],
#          'time7' : [[15, 21, 40], [15, 22, 26]],
#          'time8' : [[16, 27, 23], [16, 28, 20]],
#          'time9' : [[17, 24, 13], [17, 25, 0]],
#          'time10' : [[18, 1, 30], [18, 2, 30]],
#          'time11' : [[19, 14, 20], [19, 16, 13]],
#          'time12' : [[20, 45, 20], [20, 47, 38]],
#          'time13' : [[20, 56, 0], [20, 57, 0]],
#          'time14' : [[21, 52, 45], [21, 53, 45]],
#          'time15' : [[23, 24, 30], [23, 25, 10]],
#          'time16' : [[20, 25, 40], [20, 21, 30]]
#          }

In [ ]:
cap = cv2.VideoCapture(r"..\..\Data\TruckVideo.mp4")

#output video setting
output_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
output_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_fps = cap.get(cv2.CAP_PROP_FPS) 
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Use 'mp4v' codec for MP4 format

output_path = 'Yolov8_annotated.mp4'

out = cv2.VideoWriter(output_path, fourcc, output_fps, (output_width, output_height))

time_step = 1

#truck detection in different times
## use esc+q to switch between times
for t in times.keys():
    start_time = times[t][0][0] * 3600 + times[t][0][1] * 60  + times[t][0][2]# Convert start time to seconds
    end_time = times[t][1][0] * 3600 + times[t][1][1] * 60  + times[t][1][2] # Convert end time to seconds

    # set start time for captured video
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)

    frame_time = start_time
    
    # truck detection for time t
    while frame_time <= end_time:
        ret, frame = cap.read()
        
        if not ret:
            break

        # use model for detection
        with torch.no_grad():
            results = model(frame)

        annotated_frame = results[0].plot()

        # write number of trucks on frame
        cv2.putText(annotated_frame,f'Yolov8 Model - truck: {results[0].boxes.shape[0]}',(50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
            
        # add frame to video
        out.write(annotated_frame)
        
        # display annotated frame 
        cv2.imshow('YOLOV8', annotated_frame)

        frame_time += time_step

        # update capture start_time
        cap.set(cv2.CAP_PROP_POS_MSEC, frame_time * 1000)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break       
        
out.release()
cap.release()
cv2.destroyAllWindows()

## Detection_Video_SingleTime

In [ ]:
cap = cv2.VideoCapture(r"..\..\Data\TruckVideo.mp4")

# define start time of video for detection (secounds)
## press esc+q to stop detection
start_time = 4 * 3600 + 56 * 60  + 2

# set start time for captured video
cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)

frame_time = start_time
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break
    
    # use model for detection
    with torch.no_grad():
        results = model(frame)

    annotated_frame = results[0].plot()
    
    cv2.putText(annotated_frame,f'Yolov* Model - truck: {results[0].boxes.shape[0]}',(50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    
    # display annotated frame
    cv2.imshow('YOLOV8', annotated_frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()